In [ ]:
import pyreadr as py
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from tensorflow.python.keras.callbacks import TensorBoard
import tensorflow as tf
import numpy as np
from sklearn.metrics import multilabel_confusion_matrix,confusion_matrix,classification_report
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler,Normalizer
from tensorflow import keras
import keras_tuner as kt
import tensorflow as tf
import os
from sklearn.preprocessing import OneHotEncoder

In [ ]:
def plot_confusion_matrix(test_y, predict_y):
    # Convertir las probabilidades predichas en clases
    print(predict_y.shape)
    print(test_y.shape)
    #if len(predict_y.shape) == 1:
    #    predict_y_classes = predict_y.reshape(-1, 1)
    #else:
    #    predict_y_classes = np.argmax(predict_y, axis=1)
    
    # Calcular la matriz de confusión
    C = confusion_matrix(test_y, predict_y_classes)
    
    # Calcular la precisión y recall
    A = (((C.T) / (C.sum(axis=1))).T)
    B = (C / C.sum(axis=0))
    
    labels = [0,1,2,4,5,6,7,8,10,11,12,13,14,16,17,18,19,20]
    cmap = sns.color_palette("Greens", as_cmap=True)
    
    # Representar la matriz de confusión en formato de heatmap
    print("-" * 50, "Confusion matrix", "-" * 50)
    plt.figure(figsize=(20,20))
    sns.heatmap(C, annot=True, cmap=cmap, fmt=".2f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.savefig(f'matrices/confusion_matrix.png')
    plt.show()
    
    # Representar la matriz de precisión en formato de heatmap
    print("-" * 50, "Precision matrix", "-" * 50)
    plt.figure(figsize=(20,20))
    sns.heatmap(B, annot=True, cmap=cmap, fmt=".2f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.savefig(f'matrices/precision_matrix.png')
    plt.show()
    print("Sum of columns in precision matrix",B.sum(axis=0))
    
    # Representar la matriz de recall en formato de heatmap
    print("-" * 50, "Recall matrix", "-" * 50)
    plt.figure(figsize=(20,20))
    sns.heatmap(A, annot=True, cmap=cmap, fmt=".2f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.savefig(f'matrices/recall_matrix.png')
    plt.show()
    print("Sum of rows in precision matrix",A.sum(axis=1))


In [ ]:
#Read data
train = pd.read_csv('train.csv')
cv = pd.read_csv("cv.csv")
test = pd.read_csv("test.csv")

In [ ]:
print("Shape of the sampled train data:", train.shape)
print("Shape of the sampled test data:", test.shape)
print("Shape of the sampled CV data:", cv.shape)
print(train)

In [ ]:
# Removing faults 3,9 and 15 
tr = train.drop(train[(train.faultNumber == 3) | (train.faultNumber == 9) | (train.faultNumber == 15)].index).reset_index()
ts = test.drop(test[(test.faultNumber == 3) | (test.faultNumber == 9) | (test.faultNumber == 15)].index).reset_index()
cv_ = cv.drop(cv[(cv.faultNumber == 3) | (cv.faultNumber == 9) | (cv.faultNumber == 15)].index).reset_index()


In [ ]:
print(train)
# Resizing the train, test and cv data.
x_train = np.resize(tr,(183200,52))
x_test = np.resize(ts,(89000,52))
x_cv = np.resize(cv_,(93440,52))

In [ ]:
x_train.shape

In [ ]:
#converting the class labels to categorical values and removing unnecessary features from train, test and cv data.
y_train = tr['faultNumber']
y_test = ts['faultNumber']
y_cv = cv_['faultNumber']

encoder = OneHotEncoder()

# Ajustar y transformar los datos de salida
y_train_encoded = encoder.fit_transform(np.array(y_train).reshape(-1, 1))
y_test_encoded = encoder.fit_transform(np.array(y_test).reshape(-1, 1))
y_cv_encoded = encoder.fit_transform(np.array(y_cv).reshape(-1, 1))


In [ ]:
tr.drop(['faultNumber','Unnamed: 0','simulationRun','sample','index'],axis=1,inplace=True)
ts.drop(['faultNumber','Unnamed: 0','simulationRun','sample','index'],axis =1,inplace=True)
cv_.drop(['faultNumber','Unnamed: 0','simulationRun','sample','index'],axis =1,inplace=True)
standard_scalar = StandardScaler()
train_norm = standard_scalar.fit_transform(tr)
test_norm = standard_scalar.transform(ts)
cv_norm = standard_scalar.transform(cv_)

In [ ]:
tr.shape

In [ ]:
import os
import tensorflow as tf
import keras_tuner as kt
import numpy as np

def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(units=hp.Int('units', min_value=32, max_value=256, step=32),
                                    activation=hp.Choice('input_activation', values=['relu', 'elu'])))

    model.add(tf.keras.layers.Dense(units=hp.Int('units', min_value=32, max_value=256, step=32),
                                    activation=hp.Choice('input_activation', values=['relu', 'elu'])))
    
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(tf.keras.layers.Dense(units=hp.Int(f'layer_{i}_units', min_value=32, max_value=256, step=32),
                                        activation=hp.Choice(f'layer_{i}_activation', values=['tanh', 'sigmoid'])))
    
    model.add(tf.keras.layers.Dense(18, activation='softmax'))

    model.compile(optimizer='adamw',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def optimize_hyperparameters(train_data, train_labels, val_data, val_labels, max_epochs=10, batch_size=32):
    if not os.path.exists('hiperParametros'):
        os.makedirs('hiperParametros')

    tuner = kt.Hyperband(build_model,
                         objective='val_accuracy',
                         max_epochs=max_epochs,
                         factor=3,
                         directory='hiperParametros',
                         project_name='hiperparametrosKeras')

    tuner.search(train_data, train_labels, epochs=max_epochs, batch_size=batch_size, validation_data=(val_data, val_labels))

    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

    # Imprimir los mejores hiperparámetros
    print("Best hyperparameters found:")
    for param in best_hps.values.keys():
        print(f"{param}: {best_hps.get(param)}")

    print("\nLayer-wise Hyperparameter Information:")
    layer_index = 0
    for hp in tuner.oracle.hyperparameters.space:
        print('----------------------')
        print(list(hp.values))
       
    return best_hps


batch_size = 32
# Inicialización del optimizador
optimizer = tf.keras.optimizers.AdamW(learning_rate=0.01)
loss_fn = tf.keras.losses.mean_squared_error
mean_loss = tf.keras.metrics.Mean(name="mean_loss")
accuracy = tf.keras.metrics.Accuracy(name="accuracy")  # Nueva métrica de accuracy
metrics = [tf.keras.metrics.MeanAbsoluteError()]
# Ejemplo de uso:
best_hyperparameters = optimize_hyperparameters(train_norm, y_train_encoded.toarray(), cv_norm, y_cv_encoded.toarray())


model = build_model(best_hyperparameters)    

# Entrena el modelo
history = model.fit(train_norm, y_train_encoded.toarray(), epochs=5, batch_size=32, validation_data=(cv_norm, y_cv_encoded.toarray()))

# Evalúa el modelo
accuracy = model.evaluate(test_norm, y_test_encoded.toarray())[1]
print("Accuracy:", accuracy)


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import label_binarize

# Obtener las probabilidades predichas del modelo
predict_y_prob = model.predict(test_norm)

# Convertir las probabilidades predichas en clases
predict_y_classes = np.argmax(predict_y_prob, axis=1)

predict_y_multilabel = np.argmax(predict_y_prob, axis=1)

# Convertir las clases predichas a un formato multilabel-indicator
#predict_y_multilabel = label_binarize(predict_y_classes, classes=np.unique(y_test_encoded))

# Llamar a la función plot_confusion_matrix con las etiquetas verdaderas y las predicciones
plot_confusion_matrix(np.argmax(y_test_encoded.toarray(), axis=1), predict_y_classes)


In [ ]:
temp = pd.DataFrame(test_norm, columns=['xmeas_1',
 'xmeas_2',
 'xmeas_3',
 'xmeas_4',
 'xmeas_5',
 'xmeas_6',
 'xmeas_7',
 'xmeas_8',
 'xmeas_9',
 'xmeas_10',
 'xmeas_11',
 'xmeas_12',
 'xmeas_13',
 'xmeas_14',
 'xmeas_15',
 'xmeas_16',
 'xmeas_17',
 'xmeas_18',
 'xmeas_19',
 'xmeas_20',
 'xmeas_21',
 'xmeas_22',
 'xmeas_23',
 'xmeas_24',
 'xmeas_25',
 'xmeas_26',
 'xmeas_27',
 'xmeas_28',
 'xmeas_29',
 'xmeas_30',
 'xmeas_31',
 'xmeas_32',
 'xmeas_33',
 'xmeas_34',
 'xmeas_35',
 'xmeas_36',
 'xmeas_37',
 'xmeas_38',
 'xmeas_39',
 'xmeas_40',
 'xmeas_41',
 'xmv_1',
 'xmv_2',
 'xmv_3',
 'xmv_4',
 'xmv_5',
 'xmv_6',
 'xmv_7',
 'xmv_8',
 'xmv_9',
 'xmv_10',
 'xmv_11'])


### Partial Dependence Plot (PDP)

El gráfico de dependencia parcial PDP son gráficos que nos ayudan a entender cómo una variable específica afecta a las predicciones de nuestro modelo.

-  Para construir una PDP debemos elegir la característica de entrada que será la que queramos analizar.
- Variamos su valor, es decir, mantenemos constantes todas las demás variables y modificamos gradualmente el valor de la variable seleccionada.
- Calculamos las predicciones, es decir, para cada valor de la variable calculamos las predicciones del modelo.
- Graficamos estos resultados. Representamos en un gráfico cómo cambian las predicciones a medida que varia la variable

El eje x representa los valores de la variable (característica) seleccionada, y el eje y muestra las predicciones del modelo.

Una vez que tengamos las gráficas dibujadas podremos analizarlas para ver como se comportan. Si la curva es lineal, significa que la relación entre la variable y la predicción es aditiva. Es decir no hay muchas interacciones con otras características. Mientras que si no ocurre esto, podría significar que la característica es dependiente de otras características del conjunto.

Las PDP asumen que las variables no están correlacionadas, ya que si hay variables correlacionadas los resultados puedes resultar engañosos.

Para poder abordar las PDP de forma matemática debemos presentar algunas definiciones.

- $\textit{Modelo de regresión}$. Un modelo de regresión es un enfoque estadístico usado para modelar y analizar la relación entre una o más variables independientes (características) y una variablbe dependiente continua. La idea principal de este modelo es predecir o estimar el valor de la variable dependiente en función de los valores de las variables independientes.
- En este modelo, podemos expresar la relación entre las variables independientes y la dependiente mediante una función matemática. Esta función se usa para realizar las predicciones sobre la variable dependiente en funcionn de los valores de las variables independientes.

Existen varios tipos de modelos de regresión, cada uno de los cuales usa diferentes funciones matemáticas para modelar la relación entre las varibales. Entre ellos encontramos la regresión lineal, regresión polinómica, regresión logística y regresión de árbol de decisiones. 


Ahora supongamos que tenemos nuetra función de predicción del modelo $f(x)$ que predice una variable de respuesta $Y$ en función de $C$ características de entrada $X=(X_1, X_2, X_3,....)$

El objetivo que se busca a través de las PDP es visualizar como cambia la predicción del modelo $f(x)$ cuando variamos una o varias características de entrada $X_S$, manteniendo el resto de características constantes o promediadas.


La función de la dependencia parcial $PD_S(x_S)$ para una característica $X_S$ se define como 
$$PD_S(x_S)=E_{X_C}[f(x_S, X_C)] = \int f(x_S, x_c)d\mathbb{P}(X_C)$$

Donde 

- $PD_S(x_S)$ es la función dependencia parcial para las características $X_S$
- $E_{X_C}$ es el valor esperado sobre todas las características $X_C$ excepto $X_S$.
- $f(x)$ es el modelo de aprendizaje automático
- $x_S$ son las características para las cuales se debe trazar la función de dependencia parcia
- $x_c$ son el resto de características usadas en el modelo $f(x)$ 

Normalmente hay una o dos características en el conjunto $S$. Las características en $S$ son aquellas para las cuales queremos saber el efecto en la predicción. Los vectores $x_S$ y $x_C$ combinados forman el espacio total de características $x$

La función de dependencia parcial funciona marginalizando la salida del modelo de aprendizaje automático sobre la distribución de las características en el conjunto $C$, de modo que la función muestre la relación entre las características en el conjunto $S$ que nos interesan y el resultado predicho. Al marginalizar sobre las otras características, obtenemos una función que depende solo de las características en $S$, incluidas las interacciones con otras características.

La función $f_S(x_S)$ se estima calculando promedios en los datos de entrenamiento, también conocido como método de Monte Carlo:
$$f_S=\frac{1}{n}\sum_{i=1}^n f(x_S,x_C^{(i)})$$
Esta función nos dice, para un valor dado de las características $S$, cuál es el efecto marginal promedio en la predicción. En esta fórmula, $x_C^{(i)}$ son los valores reales de las características del conjunto de datos para las características en las que no estamos interesados, y $n$ es el número de instancias en el conjunto de datos. Para que este método funcione bien, se debe suponer que las características en $C$ no están correlacionadas con las características en $S$, de otra forma los resultados pueden verse afectados.

En el caso de la clasificación, donde el modelo de aprendizaje automático produce probabilidades, el gráfico de dependencia parcial muestra la probabilidad de cierta clase dada diferentes valores para las características en $S$. Una forma fácil de manejar múltiples clases es dibujar una línea o gráfico por clase.

El gráfico de dependencia parcial es un método global: considera todas las instancias y proporciona una declaración sobre la relación global de una característica con el resultado predicho.

In [ ]:
import shap

In [ ]:
def make_dummy_pdp(salida):
    class clf_dummy():
        def __init__(self, model, salida):
            self.model = model
            self.salida = salida

        def __call__(self, df):
            return self.model.predict(df)[:, self.salida]
    
    return clf_dummy(model, salida)

In [ ]:
# Lista de características
features = ['xmeas_1', 'xmeas_2', 'xmeas_3', 'xmeas_4', 'xmeas_5', 'xmeas_6', 
            'xmeas_7', 'xmeas_8', 'xmeas_9', 'xmeas_10', 'xmeas_11', 'xmeas_12', 
            'xmeas_13', 'xmeas_14', 'xmeas_15', 'xmeas_16', 'xmeas_17', 'xmeas_18', 
            'xmeas_19', 'xmeas_20', 'xmeas_21', 'xmeas_22', 'xmeas_23', 'xmeas_24', 
            'xmeas_25', 'xmeas_26', 'xmeas_27', 'xmeas_28', 'xmeas_29', 'xmeas_30', 
            'xmeas_31', 'xmeas_32', 'xmeas_33', 'xmeas_34', 'xmeas_35', 'xmeas_36', 
            'xmeas_37', 'xmeas_38', 'xmeas_39', 'xmeas_40', 'xmeas_41', 'xmv_1', 
            'xmv_2', 'xmv_3', 'xmv_4', 'xmv_5', 'xmv_6', 'xmv_7', 'xmv_8', 'xmv_9', 
            'xmv_10', 'xmv_11']

In [ ]:
#Se va a pintar en cada gráfica como afecta cada característica a cada uno de los ataques
#52 gráficas, con 18 curvas cada una
for feature in features:
    fig, ax = plt.subplots(figsize=(20, 25))
    for i in range(0,18):
        model_pdp=make_dummy_pdp(feature)
        print('PDP - feature ', feature)
        shap_line = shap.plots.partial_dependence(
            feature,model_pdp, temp, ice=False,
            model_expected_value=True, feature_expected_value=True, show=False, ax=ax
        )
        shap_line[0].set_label(f'Ataque {i+1}')
    ax.legend()
    fig.savefig(f"images/pdp/ataque{i}.png")
    plt.show()
    plt.close(fig)

In [ ]:
#Vamos a plotear las 2D PDP para el ataque 4
model_pdp=make_dummy_pdp(3)
attack_pairs =  [('xmeas_13', 'xmv_7'),
('xmeas_18', 'xmv_3'),
('xmeas_6', 'xmeas_7'),
('xmeas_6', 'xmeas_14'),
('xmeas_19', 'xmeas_5'),
('xmeas_1', 'xmeas_36'),
('xmeas_29', 'xmv_6'),
('xmeas_27', 'xmeas_9'),
('xmeas_34', 'xmv_8'),
('xmeas_15', 'xmeas_4'),
('xmeas_30', 'xmeas_29'),
('xmeas_23', 'xmeas_28'),
('xmeas_7', 'xmeas_30'),
('xmeas_3', 'xmv_2'),
('xmeas_24', 'xmv_1'),
('xmeas_29', 'xmeas_17'),
('xmeas_38', 'xmeas_40'),
('xmeas_11', 'xmv_4'),
('xmeas_20', 'xmv_5'),
('xmeas_26', 'xmeas_35')]

for attack1, attack2 in attack_pairs:
    print(f'PDP - attacks {attack1} and {attack2}')
    fig, ax = plt.subplots()
    shap.plots.partial_dependence(
        (attack1, attack2), model_pdp, temp, ice=False,
        model_expected_value=True, feature_expected_value=True, show=False, ax=ax
    )
    fig.savefig(f"images/pdp2D/attacks{attack1}and{attack2}.png")
    plt.show()
    plt.close(fig)


In [ ]:
#Vamos a pintar los plot de los siguientes ataques:
#Ataque 1
model_pdp_0=make_dummy_pdp(0)
for feature in features:
    print('PDP - feature ', feature)
    fig, ax = plt.subplots()
    shap.plots.partial_dependence(
        feature,model_pdp_0, temp, ice=False,
        model_expected_value=True, feature_expected_value=True, show=False, ax=ax
    )
    fig.savefig(f"images/pdp/ataque1/ataque1_{feature}.png")
    plt.show()
    plt.close(fig)

In [ ]:
#Ataque 2
model_pdp_1=make_dummy_pdp(1)
for feature in features:
    print('PDP - feature ', feature)
    fig, ax = plt.subplots()
    shap.plots.partial_dependence(
        feature,model_pdp_1, temp, ice=False,
        model_expected_value=True, feature_expected_value=True, show=False, ax=ax
    )
    fig.savefig(f"images/pdp/ataque2/ataque2_{feature}.png")
    plt.show()  # Mostrar la figura
    plt.close(fig)  # Cerrar la figura para liberar memoria

In [ ]:
#Ataque 3
model_pdp_2=make_dummy_pdp(2)
for feature in features:
    print('PDP - feature ', feature)
    shap.plots.partial_dependence(
        feature,model_pdp_2, temp, ice=False,
        model_expected_value=True, feature_expected_value=True, show=False, ax=ax
    )
    fig.savefig(f"images/pdp/ataque3/ataque3_{feature}.png")
    plt.show()  # Mostrar la figura
    plt.close(fig)  # Cerrar la figura para liberar memoria

In [ ]:
#Ataque 4
model_pdp_3=make_dummy_pdp(3)
fig, ax = plt.subplots()

for feature in features:
    print('PDP - feature ', feature)
    shap.plots.partial_dependence(
        feature,model_pdp_3, temp, ice=False,
        model_expected_value=True, feature_expected_value=True, show=False, ax=ax
    )
    fig.savefig(f"images/pdp/ataque4/ataque4_{feature}.png")
    plt.show()  # Mostrar la figura
    plt.close(fig)  # Cerrar la figura para liberar memoria

In [ ]:
model_pdp_13=make_dummy_pdp(12)

attack_pairs = [('xmeas_20', 'xmeas_21'), ('xmeas_11', 'xmv_3')]  # Reemplaza con los nombres de tus ataques
for attack1, attack2 in attack_pairs:
    print(f'PDP - attacks {attack1} and {attack2}')
    fig, ax = plt.subplots()
    shap.plots.partial_dependence(
        (attack1, attack2), model_pdp_13, temp, ice=False,
        model_expected_value=True, feature_expected_value=True, show=False, ax=ax
    )
    plt.show()
    plt.close(fig)

### Accumulated Local Effects (ALE)

Los Accumulated Local Effects (ALE) Plots emergen como una herramienta robusta para interpretar modelos complejos, proporcionando insights sobre la influencia de características individuales en las predicciones del modelo. Este método es particularmente efectivo en contextos donde las características predictivas están correlacionadas, superando algunas limitaciones de los métodos tradicionales como los Partial Dependence Plots (PDPs).

##### Importancia de los ALE Plots en el Análisis de TEP

Dado que el conjunto de datos TEP incluye múltiples variables altamente correlacionadas que afectan el rendimiento del proceso, es fundamental emplear métodos de interpretación que puedan manejar adecuadamente estas correlaciones para evitar conclusiones erróneas. Los ALE plots, que se centran en evaluar los efectos locales acumulativos de las características, proporcionan una visión clara y matizada de cómo cambios específicos en variables del proceso impactan las predicciones del modelo.

#### Desarrollo Matemático de los ALE Plots

Los ALE plots ofrecen un enfoque refinado para evaluar la importancia de las características en un modelo predictivo, especialmente útil cuando estas características están correlacionadas entre sí. Este método enfatiza el efecto local de las características, proporcionando una perspectiva detallada sobre cómo pequeñas variaciones en una característica específica influyen en las predicciones del modelo. A continuación, se proporciona una descripción ampliada de cada etapa del proceso matemático implicado en la construcción de un ALE plot, ofreciendo una interpretación profunda y precisa de las fórmulas y procesos utilizados.

- $\textit{Definición y Particionamiento de la Característica de Interés}$: Para implementar un ALE plot, primero es necesario seleccionar una característica $S$ del modelo que deseamos analizar. El rango de $S$ se divide en $k$ intervalos discretos, definidos cada uno por un par de valores $(z_{S,i−1},z_{S,i})$. Esta división permite evaluar el impacto incremental de $S$ en segmentos manejables, facilitando un análisis detallado del comportamiento de la función de predicción $f$ en diferentes partes de su dominio.
  
    - $\textit{Importancia del Tamaño de Intervalo}$: La elección del número de intervalos $k$ y su tamaño puede influir significativamente en la resolución y suavidad del ALE plot resultante. Intervalos más pequeños pueden capturar variaciones más finas en el efecto de $S$, pero también pueden llevar a una mayor variabilidad en las estimaciones de los efectos locales debido a un menor número de datos en cada intervalo.

-  $\textit{Cálculo de Efectos Locales Promedio}$: Para cada intervalo, se calcula el cambio promedio en la predicción causado por cambios dentro del intervalo. Esto se logra a través de la siguiente expresión integral:
    $$ALE_{S,i} = \int_{z_{S,i-1}}^{z_{S,i}}\left( \int_{X_C} \frac{\partial f(z_S,X_C)}{\partial z_S} dP(X_C|X_S = z_S)\right) dz_S$$

    donde:
      - La integral interna calcula el promedio de la derivada parcial de $f$ respecto a $S$, ponderada por la distribución condicional de las otras características $X_C$. Este término $\frac{\partial f(z_S,X_C)}{\partial z_S} $ representa la sensibilidad de la función de predicción a cambios infinitesimales en $S$, evaluada en cada punto $z_S$ dentro del intervalo.
        
      - Integración Externa: La integral externa acumula estos efectos locales sobre el intervalo $(z_{S,i−1},z{S,i})$, proporcionando una medida del impacto total de las variaciones en $S$ dentro de ese segmento específico. 

- $\textit{Acumulación de Efectos Locales}$ para obtener el efecto acumulativo total de la característica $S$ hasta cualquier valor $x_S$, se suman todos los efectos locales calculados para los intervalos hasta $x_S$
$$ALE_S(x_S) = \sum_{i:z_{S,i}\leq x_S} ALE_{S,i} - \text{constante}$$

- $\textit{Ajuste de la Constante}$: La constante en la fórmula es esencial para centrar los valores del ALE plot alrededor de cero. Esto se logra asegurando que el valor medio del ALE plot a lo largo de todos los valores posibles de $S$ sea cero, lo cual mejora la interpretabilidad al establecer un punto de referencia neutral para evaluar los efectos de $S$.

In [ ]:
import lime
import lime.lime_tabular
import matplotlib.pyplot as plt
import os

# Función de envoltura para el método predict del modelo Keras
def predict_proba_wrapper(data):
    return model.predict(data)

# Inicializar el explicador LIME
explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=temp.values,
    feature_names=temp.columns,
    class_names=features,  # Ajusta los nombres de las clases según tu modelo
    mode='classification'
)

# Seleccionar una instancia para explicar (por ejemplo, la primera instancia del conjunto de datos)
instance = temp.iloc[0]

# Generar la explicación para la instancia seleccionada
exp = explainer.explain_instance(
    data_row=instance,
    predict_fn=predict_proba_wrapper
)

# Dibujar y guardar la explicación
fig = exp.as_pyplot_figure()
fig.savefig(f"{output_dir}/lime_explanation_instance_0.png")
plt.show()
plt.close(fig)

# Si deseas generar explicaciones para múltiples instancias
for i in range(len(temp)):
    instance = temp.iloc[i]
    exp = explainer.explain_instance(
        data_row=instance,
        predict_fn=predict_proba_wrapper
    )
    fig = exp.as_pyplot_figure()
    fig.savefig(f"{output_dir}/lime_explanation_instance_{i}.png")
    plt.show()
    plt.close(fig)
